
# Infostrom — Gaze-Driven Selective Audio Project (IBM LinuxOne)

**Purpose:** Interactive notebook demo that simulates gaze-driven audio focus using *simple face-position logic*.
This version is tailored for **online Jupyter** environments where webcam access may be restricted. 

**Features**
- Inline frame display (no external GUI windows)
- 8 virtual monitors 
- Simple face-position gaze estimation (face center -> monitor zone)
- Visual indicator of which monitor is "active" and simulated audio status


In [ ]:
## ============================================================
# Enhanced AI Gaze Visualization Dashboard (Keyboard + Layout Polish)
# ============================================================
import cv2, numpy as np, time, asyncio
import ipywidgets as widgets
from ipycanvas import Canvas
from IPython.display import display
from math import sin

# ============================================================
# State
# ============================================================
class State:
    def __init__(self):
        self.running = False
        self.speed = 50        # ms/frame
        self.num_monitors = 8
        self.gaze_pattern = "mouse"
        self.frame = np.zeros((400, 640, 3), np.uint8)
        self.monitor_idx = None
        self.trail = []
        self.mouse_x, self.mouse_y = 0.5, 0.5
        self.target_x = 0.5
        self.keyboard_x = 0.5
        self.mouse_active = False
        self.fps = 0

state = State()

# ============================================================
# Logging (final version – neat + scrollable)
# ============================================================
log_history = []

log_output = widgets.Output()
log_container = widgets.Box(
    [log_output],
    layout=widgets.Layout(
        border="1px solid #444",
        height="140px",
        overflow_y="scroll",
        overflow_x="hidden",
        width="100%",
        background="#111",
        padding="6px",
        color="#ddd",
        font_size="12px",
        flex_flow="column nowrap"
    )
)

def log_message(msg):
    """Append message safely to the log box."""
    log_history.append(msg)
    # Limit to last 50 entries
    if len(log_history) > 50:
        del log_history[0]
    log_output.clear_output(wait=True)
    with log_output:
        for line in log_history:
            print(line)




# ============================================================
# Canvas + Events
# ============================================================
canvas = Canvas(width=640, height=400)

def handle_mouse_move(x, y):
    if state.gaze_pattern == "mouse":
        state.target_x = x / 640
        state.mouse_y = y / 400
        state.mouse_active = True
canvas.on_mouse_move(handle_mouse_move)

def handle_key_down(event):
    k = event["key"]
    if state.gaze_pattern == "keyboard":
        if k == "ArrowLeft":
            state.keyboard_x = max(0.0, state.keyboard_x - 0.05)
        elif k == "ArrowRight":
            state.keyboard_x = min(1.0, state.keyboard_x + 0.05)
        elif k.isdigit() and 1 <= int(k) <= state.num_monitors:
            idx = int(k) - 1
            state.keyboard_x = (idx + 0.5) / state.num_monitors
            log_message(f"🎯 Jumped to Monitor {int(k)}")

canvas.on_key_down(handle_key_down)

def show_frame():
    img = cv2.cvtColor(state.frame, cv2.COLOR_BGR2RGB)
    canvas.put_image_data(img)

# ============================================================
# UI Controls
# ============================================================
start_btn = widgets.Button(description="▶ Start", button_style="success")
pause_btn = widgets.Button(description="⏸ Pause", button_style="warning")
stop_btn  = widgets.Button(description="⏹ Stop",  button_style="danger")
reset_btn = widgets.Button(description="🔁 Reset", button_style="info")

speed_slider   = widgets.IntSlider(value=int(state.speed), min=10, max=200, step=5, description="Speed (ms)")
monitor_slider = widgets.IntSlider(value=state.num_monitors, min=2, max=8, step=1, description="Monitors")
pattern_dd     = widgets.Dropdown(
    options=["mouse", "sine", "linear", "keyboard", "zigzag"],
    value="mouse",
    description="Mode"
)

status_text = widgets.HTML("<b>Status:</b> 💤 Idle")
status_light = widgets.HTML("<div style='width:12px;height:12px;border-radius:50%;background:#ccc;display:inline-block;margin-right:6px;'></div>")

# ============================================================
# Status Update
# ============================================================
def update_light():
    color = "#28a745" if state.running else "#dc3545"
    status_light.value = f"<div style='width:12px;height:12px;border-radius:50%;background:{color};display:inline-block;margin-right:6px;'></div>"

def update_status():
    update_light()
    icon = {
        "mouse": "🖱️",
        "keyboard": "⌨️",
        "sine": "🌊",
        "linear": "➡️",
        "zigzag": "⚡"
    }.get(state.gaze_pattern, "👁️")
    badge = f"🟩 Monitor {state.monitor_idx}" if state.monitor_idx is not None else "🟦 Idle"
    status_text.value = f"<b>Status:</b> {'🟢 Running' if state.running else '🔴 Stopped'} | {badge} | Mode: {icon} {state.gaze_pattern.title()} | FPS: {int(state.fps)}"

# ============================================================
# Animation Loop
# ============================================================
async def simulate_async():
    log_message("🟢 Simulation started.")
    state.running = True
    last_time = time.time()
    state.trail.clear()

    while state.running:
        mode = state.gaze_pattern
        if mode == "mouse":
            gx = state.target_x
        elif mode == "keyboard":
            gx = state.keyboard_x
        elif mode == "sine":
            gx = (sin(time.time() * 0.7) + 1) / 2
        elif mode == "linear":
            gx = (time.time() * 0.25) % 1.0
        elif mode == "zigzag":
            t = (time.time() * 0.5) % 2
            gx = t if t <= 1 else 2 - t
        else:
            gx = 0.5

        # Smooth motion
        state.mouse_x += (gx - state.mouse_x) * 0.25

        # Grid layout 2x4
        cols = 4
        rows = int(np.ceil(state.num_monitors / cols))
        monitor_w = 640 // cols
        monitor_h = 400 // rows
        total = state.num_monitors

        m_idx = int(state.mouse_x * total)
        m_idx = min(m_idx, total - 1)
        state.monitor_idx = m_idx

        # Create frame
        f = np.zeros_like(state.frame)
        pulse = (sin(time.time() * 4) + 1) / 2
        for i in range(total):
            r, c = divmod(i, cols)
            x1, y1 = c * monitor_w, r * monitor_h
            x2, y2 = x1 + monitor_w, y1 + monitor_h
            color = (int(80 + 150 * pulse), int(255 * pulse), 0) if i == m_idx else (70, 70, 70)
            cv2.rectangle(f, (x1, y1), (x2, y2), color, 2)
            cv2.putText(f, f"{i+1}", (x1+8, y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)

        # Ball trail
        cx = int(state.mouse_x * 640)
        cy = 200 + int(sin(time.time() * 3) * 60) if mode != "mouse" else int(state.mouse_y * 400)
        state.trail.append((cx, cy))
        if len(state.trail) > 12:  # shorter trail
            state.trail.pop(0)

        for i, (tx, ty) in enumerate(state.trail):
            a = i / len(state.trail)
            cv2.circle(f, (tx, ty), 5, (0, int(255*a), int(255*(1-a))), -1)
        cv2.circle(f, (cx, cy), 10, (0, 255, 255), -1)

        now = time.time()
        state.fps = 1 / (now - last_time) if now > last_time else 0
        last_time = now

        state.frame = f
        show_frame()
        update_status()
        await asyncio.sleep(state.speed / 1000.0)

    log_message("🛑 Simulation stopped.")

# ============================================================
# UI Handlers
# ============================================================
def start_clicked(b):
    if not state.running:
        asyncio.create_task(simulate_async())
    else:
        log_message("⚠️ Already running.")

def pause_clicked(b): state.running=False; update_status()
def stop_clicked(b):  state.running=False; state.monitor_idx=None; update_status()
def reset_clicked(b):
    state.running=False
    state.trail.clear()
    state.frame = np.zeros((400, 640, 3), np.uint8)
    state.mouse_x = state.mouse_y = state.target_x = state.keyboard_x = 0.5
    show_frame(); update_status(); log_message("🔁 Reset complete.")

def on_speed_change(ch): state.speed = ch["new"]
def on_pattern_change(ch):
    state.gaze_pattern = ch["new"]
    log_message(f"🔄 Mode changed to {ch['new'].title()}")
    update_status()
def on_monitor_change(ch):
    state.num_monitors = ch["new"]
    log_message(f"🖥️ Monitors set to {ch['new']}")

# Wire buttons
for w, fn in [(start_btn,start_clicked),(pause_btn,pause_clicked),(stop_btn,stop_clicked),(reset_btn,reset_clicked)]:
    w.on_click(fn)
speed_slider.observe(on_speed_change,"value")
pattern_dd.observe(on_pattern_change,"value")
monitor_slider.observe(on_monitor_change,"value")

# ============================================================
# Layout
# ============================================================
controls = widgets.HBox([start_btn, pause_btn, stop_btn, reset_btn, status_light])
config   = widgets.HBox([speed_slider, monitor_slider, pattern_dd])

dashboard = widgets.VBox([
    widgets.HTML("<h3 style='text-align:center;color:#ddd;'>👁️ AI Gaze Visualization Dashboard</h3>"),
    controls, status_text,
    widgets.Box([config], layout=widgets.Layout(justify_content="center")),
    widgets.Box([canvas], layout=widgets.Layout(justify_content="flex-start", margin="0 0 0 40px")),  # shifted left
    widgets.HTML("<h4 style='color:#ddd;'>📋 Event Log</h4>"),
    log_output
])

display(dashboard)
show_frame()
update_status()
log_message("🎮 Ready! Choose mode then press ▶ Start.")
